In [1]:
//import packages
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}

Intitializing Scala interpreter ...

Spark Web UI available at http://de52330afafb:4040
SparkContext available as 'sc' (version = 2.4.3, master = local[*], app id = local-1559550736553)
SparkSession available as 'spark'


import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}


In [16]:

// Load wrangled datasets from Phase 2: part1.csv; part2.csv; part3.csv
//Combine the dataframes into one

val creditRiskdf1 = spark.read.option("header","true").csv("part1.csv")
val creditRiskdf2 = spark.read.option("header","true").csv("part2.csv")
val creditRiskdf3 = spark.read.option("header","true").csv("part3.csv")

creditRiskdf1: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf2: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf3: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [17]:
val creditRiskdf1and2 = creditRiskdf1.union(creditRiskdf2)
val creditRiskdf = creditRiskdf1and2.union(creditRiskdf3)

creditRiskdf1and2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]
creditRiskdf: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 283 more fields]


In [4]:

// check that the rows have been aggregated correctly

println("Total data rows in loaded dataframes:"+(
    creditRiskdf1.count()+creditRiskdf2.count()+creditRiskdf3.count()))

println("Total data rows in combined dataframe:"+creditRiskdf.count())

2019-06-03 08:32:40,763 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
Total data rows in loaded dataframes:183875
Total data rows in combined dataframe:183875


In [20]:
//Drop the inconsistent value
val incon_droplist = List("previous_loans_DAYS_FIRST_DUE_mean",
                      "previous_loans_SELLERPLACE_AREA_min",
                      "previous_loans_DAYS_FIRST_DUE_sum")

val df = creditRiskdf.drop(incon_droplist:_*)

incon_droplist: List[String] = List(previous_loans_DAYS_FIRST_DUE_mean, previous_loans_SELLERPLACE_AREA_min, previous_loans_DAYS_FIRST_DUE_sum)
df: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [21]:

val creditRiskdf = df

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [22]:

creditRiskdf.cache()

2019-06-03 08:37:23,490 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.


res5: creditRiskdf.type = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [23]:
//Split dataframe into training, test and validation
var Array(train_df,test_df,validation_df)=creditRiskdf.randomSplit(
    Array(0.7,0.2,0.1))

train_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
test_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]
validation_df: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [9]:
train_df.cache()

res2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [EXT_SOURCE_2: string, EXT_SOURCE_3: string ... 280 more fields]


In [11]:
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [12]:
var features = creditRiskFeatures.filter(!_.contains("TARGET"))

features: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CREDIT_mean,...

In [13]:
//Set all the features as assembler for logistic regression
val assembler = new VectorAssembler()
  .setInputCols(features)
  .setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_7fa25b6d3691


In [28]:
//Convert data type to float to enable logistic regression.
val creditRiskFeatures = creditRiskdf.columns
for (colName<-creditRiskFeatures){
     |   train_df=train_df.withColumn(colName,col(colName).cast("Float"))
     | }

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, client_installments_NUM_INSTALMENT_VERSION_mean_max, bureau_DAYS_CR...

In [29]:
train_df.printSchema()

root
 |-- EXT_SOURCE_2: float (nullable = true)
 |-- EXT_SOURCE_3: float (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: float (nullable = true)
 |-- DAYS_BIRTH: float (nullable = true)
 |-- AMT_CREDIT: float (nullable = true)
 |-- AMT_ANNUITY: float (nullable = true)
 |-- DAYS_EMPLOYED: float (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: float (nullable = true)
 |-- bureau_DAYS_CREDIT_max: float (nullable = true)
 |-- DAYS_ID_PUBLISH: float (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: float (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: float (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: float (nullable = true)
 |-- DAYS_LAST_PHONE_CHANGE: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_max: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_mean: float (nullable = true)
 |-- DAYS_REGISTRATION: 

 |-- previous_loans_NAME_GOODS_CATEGORY_Mobile_count_norm: float (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_OVERDUE_mean: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Construction_count_norm: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_SCOFR_count_norm: float (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Furniture_count_norm: float (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_norm_max: float (nullable = true)
 |-- previous_loans_NAME_GOODS_CATEGORY_Construction Materials_count_norm: float (nullable = true)
 |-- client_bureau_balance_STATUS_1_count_norm_sum: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_Cash Street: middle_count_norm: float (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS other with interest_count_norm: float (nullable = true)
 |-- client_cash_CNT_INSTALMENT_min_min: float (nullable = true)
 |-- previous_loans_CODE_REJECT_REASON_VERIF_count_norm: float (nullable 

In [26]:
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.3)
  .setElasticNetParam(0.8)
  .setFeaturesCol("features")   // setting features column
  .setLabelCol("TARGET") //Setting label column

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_21dbe4b8b92b


In [ ]:
val pipeline = new Pipeline().setStages(Array(assembler,lr))
val lrModel = pipeline.fit(train_df)

In [ ]:
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")